In [1]:
import os
import urllib.request
import zipfile
import tarfile
import progressbar

In [2]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
import shutil
from tqdm import tqdm

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
# プログレスバーを表示させる関数
pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

In [5]:
# カレントdirを確認
getcwd()

'c:\\Users\\yuuki\\Kaggle\\object_detection_try\\voc'

In [6]:
# データを保存するdirを作成
data_dir = "./data/"
os.makedirs(data_dir, exist_ok=True)

In [7]:
# voc2012のデータをダウンロード
# ダウンロード先のurl
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"

# ダウンロードするデータが保存される先
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar")

# もし保存先がなかったらダウンロードするというコード
# 指定されたパスにファイルが存在しない場合、処理を実行します。
if not os.path.exists(target_path):
    # 指定されたURLからファイルをダウンロードします。
    urllib.request.urlretrieve(url, target_path, show_progress)
    # ダウンロードしたファイルを解凍し、指定されたディレクトリに展開します。
    tar= tarfile.TarFile(target_path)
    tar.extract(data_dir)
    tar.close()

## tarfileが展開されないのでguiでdatadirで展開を行った

In [8]:
target_path

'./data/VOCtrainval_11-May-2012.tar'

## VOCのXMLフォーマットからYOLOのTXTフォーマットに変換

In [9]:
# vocのクラスの確認
# urlのサイトから確認
classes = ['aeroplane', 'bicycle', 'bird', 'boat',
        'bottle', 'bus', 'car', 'cat', 'chair',
        'cow', 'diningtable', 'dog', 'horse',
        'motorbike', 'person', 'pottedplant',
        'sheep', 'sofa', 'train', 'tvmonitor']

In [10]:
def getImagesInDir(dir_path):
  """
  指定されたディレクトリ内のすべての画像ファイルのパスを取得する関数

  Args:
      dir_path (str): 画像ファイルを含むディレクトリのパス

  Returns:
      list: 拡張子付きの画像ファイルパスのリスト (例: ".jpg")
  """

  image_list = []
  for filename in glob.glob(dir_path + '/*.jpg'):
    image_list.append(filename)

  return image_list


def convert(size, box):
  """
  画像サイズに基づいてバウンディングボックス座標を正規化する関数

  Args:
      size (tuple): 画像の幅と高さを含むタプル (w, h)
      box (tuple): バウンディングボックス座標を含むタプル (xmin, xmax, ymin, ymax)

  Returns:
      tuple: 正規化されたバウンディングボックス座標を含むタプル (x_center, y_center, width, height)
  """

  dw = 1. / size[0]  # 幅の正規化係数
  dh = 1. / size[1]  # 高さの正規化係数

  x_center = (box[0] + box[1]) / 2.0 - 1  # 中心 x 座標 (正規化)
  y_center = (box[2] + box[3]) / 2.0 - 1  # 中心 y 座標 (正規化)
  width = box[1] - box[0]  # 元の幅
  height = box[3] - box[2]  # 元の高さ

  x_center = x_center * dw  # 正規化された中心 x
  width = width * dw  # 正規化された幅
  y_center = y_center * dh  # 正規化された中心 y
  height = height * dh  # 正規化された高さ

  return (x_center, y_center, width, height)

In [11]:
def convert_annotation(dir_path, output_path, image_path):
  """
  画像の注釈を XML 形式から YOLO 形式のテキストファイルに変換する関数

  Args:
      dir_path (str): XML 注釈ファイルを含むディレクトリのパス。
          ファイル名は拡張子なしで画像ファイル名と一致する必要があります。
      output_path (str): 変換されたテキストファイルを保存するディレクトリのパス
      image_path (str): 注釈を変換する対象の画像ファイルのパス

  Returns:
      None
  """

  basename = os.path.basename(image_path)  # 画像パスからファイル名を取得
  basename_no_ext = os.path.splitext(basename)[0]  # ファイル名から拡張子を除去

  # XML 注釈ファイルを開く
  in_file = open(dir_path + '/' + basename_no_ext + '.xml')
  tree = ET.parse(in_file)
  root = tree.getroot()

  # XML から画像サイズを取得
  size = root.find('size')
  w = int(size.find('width').text)
  h = int(size.find('height').text)

  # 出力用テキストファイルを開く
  out_file = open(output_path + basename_no_ext + '.txt', 'w')
  for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [12]:
# 学習用データ準備
data_path = "data/VOCdevkit/VOC2012/"
imgpath_template = os.path.join(data_path, 'JPEGImages', '%s.jpg')
annopath_template = os.path.join(data_path, 'Annotations', '%s.xml')

train_id_names = os.path.join(data_path + 'ImageSets/Main/train.txt')
val_id_names = os.path.join(data_path + 'ImageSets/Main/val.txt')

# 学習用データ
train_img_list = []
train_anno_list = []
for line in open(train_id_names):
    file_id = line.strip()
    train_img_list.append(f"{data_path}JPEGImages/{file_id}.jpg")
    train_anno_list.append(f"{data_path}Annotations/{file_id}.xml")

# 評価用データ
val_img_list = []
val_anno_list = []
for line in open(val_id_names):
    file_id = line.strip()
    val_img_list.append(f"{data_path}JPEGImages/{file_id}.jpg")
    val_anno_list.append(f"{data_path}Annotations/{file_id}.xml")

In [15]:
dst_dir = "dataset"
os.makedirs(dst_dir, exist_ok=True)
os.makedirs(f"{dst_dir}/train/images/", exist_ok=True)
os.makedirs(f"{dst_dir}/val/images/", exist_ok=True)
os.makedirs(f"{dst_dir}/train/labels/", exist_ok=True)
os.makedirs(f"{dst_dir}/val/labels/", exist_ok=True)

In [16]:
# # .jpgと.xmlをdst_dirの中のサブフォルダにコピー

for f in tqdm(train_img_list):
    file_id = os.path.basename(f)
    shutil.copy(f, f"{dst_dir}/train/images/{file_id}")
for f in tqdm(train_anno_list):
    file_id = os.path.basename(f)
    shutil.copy(f, f"{dst_dir}/train/labels/{file_id}")
for f in tqdm(val_img_list):
    file_id = os.path.basename(f)
    shutil.copy(f, f"{dst_dir}/val/images/{file_id}")
for f in tqdm(val_anno_list):
    file_id = os.path.basename(f)
    shutil.copy(f, f"{dst_dir}/val/labels/{file_id}")

100%|██████████| 5823/5823 [00:05<00:00, 990.85it/s] 


In [17]:
getcwd()

'c:\\Users\\yuuki\\Kaggle\\object_detection_try\\voc'

In [22]:
# .xmlから.txtに変換
label_dirs = [f"{dst_dir}/train/labels", f"{dst_dir}/val/labels"]
image_dirs = [f"{dst_dir}/train/images", f"{dst_dir}/val/images"]

In [23]:
label_dirs

['dataset/train/labels', 'dataset/val/labels']

In [24]:
image_dirs

['dataset/train/images', 'dataset/val/images']

In [26]:
cwd = getcwd()
cwd

'c:\\Users\\yuuki\\Kaggle\\object_detection_try\\voc'

In [28]:
# xmlからtxtファイルを作成するコード
for image_dir_path, label_dir_path in list(zip(image_dirs, label_dirs)):
    full_image_path = cwd + "/" + image_dir_path
    full_label_path = cwd + "/" + label_dir_path
    # print(full_image_path)
    # print(full_label_path)

    # xmlをtextに変えたファイルの出力先
    output_path = full_label_path + '/'
    # .jpgのpathを取得
    image_paths = getImagesInDir(full_image_path)
    # 
    with open(full_label_path + '.txt', 'w') as list_file:
        for image_path in image_paths:
            list_file.write(image_path + '\n')
            convert_annotation(full_label_path, output_path, image_path)

        print("Finished processing: ")

Finished processing: 
Finished processing: 


In [29]:
# xmlファイルをlabelsフォルダから削除する
def delete_xml_files(folder_path):
  """
  指定されたフォルダからすべてのXMLファイルを削除する関数

  Args:
      folder_path (str): XMLファイルを含むフォルダのパス

  Returns:
      None
  """

  for filename in os.listdir(folder_path):
    if filename.endswith('.xml'):
      os.remove(os.path.join(folder_path, filename))


In [31]:
# delete_xml_filesの実行
for label_dir in label_dirs:
  folder_path = label_dir
  # print(folder_path)
  delete_xml_files(folder_path)